<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Nama: Derfansyah Guswiranata Kurnia
* Identitas Dicoding : dgkurnia
* email dicoding : weerakurnia@gmail.com
* Alamat: Jalan Malaka Hijau No 39 Perumahan Malaka Country Estate

# Tahap awal perpustakaan

## Pembaruan

In [1]:
#perkembangan google colab
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [2]:
#request dan pandas
!pip install requests pandas

In [3]:
#instalasi sastarawi
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.0 MB/s eta 0:00:00


In [4]:
#instalasi wordcloud
!pip install wordcloud

In [5]:
#instalasi backoff
!pip install backoff aiohttp

## Impor Perpustakaan

In [6]:
#akses ke google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#instalasi perpustakaan
import requests
import json
import pandas as pd
import csv
#ke manajemen

In [8]:
#manajemen data dasar
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt #jika ada tanggal
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d
#ke sastawi

In [9]:
#persiapan 'multiprocessing'
from multiprocessing import Pool
#Seksi Io dan lain-lain
import os
from io import StringIO
#pemrograman paralel
import concurrent.futures #untuk pemrograman paralel
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
#ke sastrawi

In [10]:
#NITK
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.
from nltk.corpus import stopwords  # Mengimpor modul stopwords dari pustaka NLTK.
from nltk.tokenize import word_tokenize  # Mengimpor fungsi word_tokenize dari modul tokenize dalam pustaka NLTK.

#impor wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
#ke sastrawi

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
#perpustakaan untuk klasifikasi
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

# Persiapan ekstraksi situs

## Pendaftaran situs

In [109]:
#daftar url
urllist = ["https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review", "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-nessie-dark-green-s/review",
"https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gruney-katun-red-m/review", "https://www.tokopedia.com/erigo/kaos-erigo-t-shirt-oversize-antelope-cotton-combed-black-m/review"]
#ke fragmentasi

In [110]:
#fragmentasi url
[urla,urlb,urlc,urld,urle,urlf] = urllist
#ke proses ekstraksi informasi

## Proses ekstraksi

In [47]:
#situs A
turla = 'https://gql.tokopedia.com/graphql/productReviewList'
heada = {'accept': '*/*' ,
   'accept-language': 'en-US,en;q=0.9' ,
   'bd-device-id': '7461986961269081617' ,
   'content-type': 'application/json' ,
   'cookie': '_UUID_NONLOGIN_=e66ed06832acf756d426a493bf7bac26; _UUID_NONLOGIN_.sig=JpwY5Akhm0H8xr6doCJFqYqnNjM; _SID_Tokopedia_=NwrKP0GhH27USJ0-Kd_bzQmNQmkL6fJZ_hUy7px9yYDRZxXSMZ9QUiGnjxgVKNqDa3dKAKRs4PNb9KXl1lOu7EO1g8VHe6DT5qnKcwrmTq6LiJqkaQIVgDk7fGkvVozC; DID=5fc0d7cafdcac8e6f2b8b9cc5b7d62a618027955df14c273f58635e60983697a254bd4679379abc68b6565430baabd72; DID_JS=NWZjMGQ3Y2FmZGNhYzhlNmYyYjhiOWNjNWI3ZDYyYTYxODAyNzk1NWRmMTRjMjczZjU4NjM1ZTYwOTgzNjk3YTI1NGJkNDY3OTM3OWFiYzY4YjY1NjU0MzBiYWFiZDcy47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; ak_bmsc=94D1DCC12F6AF478887548E9837200FB~000000000000000000000000000000~YAAQ3i2pyqNTOWCUAQAAklTfgxrwDEn0XJX2byHL/KDtwxOnz0wDQ3WSTUw7KPiuWfUUSFYOULXksJd1NJ5McYwwmS68dg26nLjKNnh9F2yS8R9hiVZ7MJYUQYu6aelqotQUNSBBRv3OrLrWXevhmyD5xTz7K/eUXo6sqxWM7ZtobnCT9IYJceNWMr2/suwIJVeQeKFswI0Wy16DlcWcO/rSokBHAedyxW81Rj/vrhGk1h2EkVxKbGjNQHOizk+TF3TWxHgEEdktw/OLLmnuPzW+IfVPL3V1rxSYqyu04eSXsc8iFC0Iwf9QFMTjXB2mjBTjj6VgtNJq0sbEJ0ghA2n/Dbfn9eaJ4LBlXlse+UVXzOCMeBjsk9lGywcV3RatT7BAmSYPFh9FTesrWA2V19S5kYnn7K3r2sxiYOJUeyKXe2tiredmK+24EE4y+u4E3ZpPBV/zpKG+hD2Iccm7; _gcl_au=1.1.1215995700.1737379239; _gid=GA1.2.1415196745.1737379240; AMP_TOKEN=%24NOT_FOUND; _tt_enable_cookie=1; _ttp=jHsfuWBzE3VgTMaYbVweVb1J_4h.tt.1; TOPATK=SSY09UDGR6eHNQR9og9-aQ; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6IjQ0MzljOWJmLTBmNWQtNTAxMy1hMzJkLTg3OTg3YTk3MzNmMyIsImNyZWF0ZWQiOjE3MzczNzkyNTIxMTYsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=Lz2FAkosBZ+4oXve1QBo9q+9+hv+7bEG9RpiIq9C7QGKm0pZ; aus=1; _CASE_=247d3e163b7d657d696b6a6a6e6c676b7d737d3c163b7d656e6867737d3b163b7d656d6d666d737d330a2f3b7d657d6d6f6d6a726f6e726d6f0b6d6f656d6e656d68746f68656f6f7d737d333e2b7d657d7269716d6d6b686f69667d737d333d337d657d323e333e343e7f3c30312b2d267f3a2c2b3e2b3a7f28362d3e7d737d333031387d657d6e6f6971666b6f6e686a7d737d2f1c307d657d6e6c6b696f7d737d2c163b7d656e6e6a6c6f6a686c737d2c0b262f3a7d657d30303c7d737d28163b7d656f737d28372c7d657d04027d737d28372c2c7d657d04027d22; _UUID_CAS_=d3b6a3eb-49e3-4a08-aea3-1b901c5edaea; _fbp=fb.1.1737380279403.789036172559542773; bm_sz=2F6996B87B5DF943A5AFCC637B52DE89~YAAQ3i2pyl2sOmCUAQAAJDcDhBoJna/+Pi9+9FYtVeHxERiNix50LL8tsfebKrx6XfBNH8IynBZOLh/p8zBAkCySWaHYdLjtbCQ/YLHlT6ELyZ38UB4KBgareKabnYiKrrsOkZlIWSXvXblVMUxRDYGMUJLQg4oQbpwkGHHN1g48qJyikkRxQnx8Sx1uC7C1NlWLI4wFIDetavq+E0Is6gHeFODNYROVUBE6rl+L92bKt8/ktpg+rk6xFbtA5dp//z4auKN9xu6CNone8D6fq51AnUmNLYmO2DHz4myqVigwAYbPx5CazWzCf7Swm2Owq/baJukEVcfpWtFf3zPiW1kPYAkLiy6RQM5BBvpNoA6xb8yX0/eCpHfEdChFIOHCm+4L7r9/UoyQXheoFGTS/rK3ZhrW5kiMvHsH9aCyR3cHNrL/rq5FqANMm7hRQPZzVzwCJ/UWYkU27MT7CrRKUzBKKumz518IQhuoKdbm5RamMwwJheYeQXLvhb6ihrMPFeVZcZf652MoY/A=~4403521~4273464; _abck=2795332B26E74C8E0CF18652596E7B31~0~YAAQ3i2pymOsOmCUAQAAmTcDhA1V9XSqU4VNslOTNJ6jixnQwM6L9TcOj/8AB/b+Q6IAnHcFLE/bWbR837Bk4vQ47T51LjZn/5qM9yJNtiO2505U5Ogc1UwYGHuKEPZWypEDF41mji9disNcwia6r1kzG+xBm/kJGtc7VH15Zei6awmXXES1wISqJiHWlk7BFaGWJ7AL71RvHDdh3ZeZIuRaC22xLJN4y7rOgz3oOKHRdh1YTVX+KU4ss8ABcjOEDMnwVLjwipHFqzXiUTtxS06K4yrci53+gvIdLd0aTsxcLZ9aIe5wHbeiJcW/sd8k05ROjNVD728AsgrUHrFAYomFwx3etX7raiQk8H1qYcpeJFcCiactzIC/X5oFupLu09kGjChSHZ3CkrC/TJxm1jBoAcgpAlveqpbPcsvXGE043mvIggSbY70X35082isWOJdhHL77oxfAv2i2i/nigQnze+9m7LrnZn13Jxpljl88sqTbUEb0Oue1OrTsA2FlGllm3SZWCbJoJ3ra7eTCaa7nrsIEyI0S9xReHWtli6NrMSMVqxcv/tKahdN1b3vvJrvnh5DQ9sCytJ0j6d3EA/gRkqm5DFQclJv/Zj4MEuyCRi4KcrHzQRS8Jw664d1N+Ad+MoeY5kqwpqUeTXV5EBTqOGS1wph5EOMWRoL9ZNXZ73iXKHThrP7M34Z46BvPvU2iHrGfeTgZNQWXK58=~-1~-1~-1; bm_sv=CCA5A936D5A930DB67816034A2DAC31B~YAAQ9S2pym4u622UAQAAtTwDhBrnJGZ7Jzo57FhlcO3NOvH2mBxoViK5QLiH6A05dODlIUTnmS3YC+GdYTWHE1UVYd3n/QmjKrDENUI4wOH4LoJlgQG04oKIL/q35U/UvpDLHW5u0CajNt3vpaHpS68BpezpxaSXXzB4IqT3337saHX8MaTsX+owTMs509miTATvFX7sBb2CqY6Gb2IHQ8KDWGDYc0gAS3I7IF3vBQTx9NUbIA13/rxpXgSRZxG62fkUkQ==~1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.82cd0c9c95d5e4aacb8928d59402da3d.1737381620878.1737381620878.1737381772505.2%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.83214773cebbd44c69bc75a5fb8f653c.1737379252363.1737379252363.1737379280151.1%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737379240.1.1.1737381772.60.0.0; _ga=GA1.1.691029263.1737379240; webauthn-session=401028bc-0678-4d07-b65a-d87c9f94f490',
   'dnt': '1' ,
   'origin': 'https://www.tokopedia.com',
   'priority': 'u=1, i',
   'referer': urla,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
   'sec-ch-ua-mobile': '?0',
   'sec-ch-ua-platform': '"Windows"',
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors',
   'sec-fetch-site': 'same-site',
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
   'x-price-center': 'true',
   'x-source': 'tokopedia-lite',
   'x-tkpd-lite-service': 'zeus',
   'x-version': '1c2c0dd'}
#operator A
quea = f'[{{"operationName":"productReviewList","variables":{{"productID":"7576252307","page":1,"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n      }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n      }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n      }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n      }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n      }}\\n      badRatingReasonFmt\\n      __typename\\n    }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n    }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'

In [79]:
#situs B
turlb ='https://gql.tokopedia.com/graphql/productReviewList'
headb = {'accept': '*/*' ,
   'accept-language': 'en-US,en;q=0.9' ,
   'bd-device-id': '7462131835184662033' ,
   'content-type': 'application/json' ,
   'cookie': '_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; AMP_TOKEN=%24NOT_FOUND; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQBJ9OcGCHKWmUAQAAD3L+hRp/QT8vjTsgrbBZ5HS3EOHVKh5GKeTybqhtIBGY/XVGwcatr669ugRz0hepTftu/NbWvUO2Kj7Q9Z01mC9KIDWhN/QNRGiMxbWynQ+qYvjGkKzrlgCaIC7DL45uUd1tB/XI95GaZ4259TRvGsKmOVaA34zhkM+/oiLiPc6oY8rR1/6yTQNlh1zczaE93WjKGosbUdkE2SYt659RWL5FMwQCJ8HGGsj/VI+6dcuzZVwklA==~1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQCOnOF6exDoWUAQAA54D+hRo7rcYdyWKCymwMXzaP7HwNEPgrDEuRSaF519576hoiPLmx0FPlK6IIWkFg5LJAS+6743VHjN+bxYYnezxjHSGsFnsRa/kSij0lP9iETUT7VRD2BFDEbiiCYaS13AHEaADsYxf6bB4i73YmaG6e0tLw/ox92OJUliJ3WsB2zpVOmyTRTCJVh7OFOHl/onLXDBJpGo9WTPjifXMCBvcSStbld0kQMXuT7rwOocRclOOoQ2Ltp7UHA3R+X5vtG+LnYzg/x8SdSnWVXDLv5uLy5v4fY1qihi2PYTxRlt3whXXxdlN8t8X5/mLqf2bMQuJj3Y1u3kmrP5miU9SpZfskrDGgU8QeVCXycK4Vv5eL0lzjhVPI++BlCfhvt45IXH7NoABweLDEqhzN2g+gsI/Hq99pZCLAvEEJMo86IQb7pqo++yq5xm1+PwaIjWFIuRQB+bsZ1IkR3BMTh2v1ke1P4oTT/54sBKLuz0c39NzNEBKkk2uJuBTneIpE1/V6j8AqcAZwhMHwCAMe8GvwRk3S6YSz4lGJgh3wGhbmUnYI~3622200~3552837; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQ5i2pyqIHG2mUAQAAhh0Phg3gcdyujPqux1AKOb2fB96tgAMucrNLZQWocTsgI+tOxBnoDsDJmksfeDUyNbGpPZrOFzbJ/fUxTFNPxtqGhIjrfVHekV9WcwFsk8oV81DFOEvxJ7TZMXkplEs129HNAcwo3bEdCALjW3nT3t/8yGQRl5PHe6eXoHX4kIQXYGPZkoFbDQjIQwaYM4T0lrqX+Im+msBvP7o6WBrtGfY7rA61OshvcobQ/MU7iUaMYA6R1NHQsQ0Zs/Ev79RPFF04oq4eEnqrgKBkzaAdjJplPv57xJ2VTIbEGLwke/eEWDN+lSLVGZl+vKUnz0TkQtsEKhDDwuFflOgYKyngQSybgyzWs0ErbsZFEeSudgn/wvS3phG4qoOdlhrq18yYEh7xMZQNUWx/YzVbO6dMQ1WqWHP9Cjo/z7w7rAbvxSAe8t6VvMMBkiiAgPYWhuYfQzIC1OyNvcRKOso/5/tFuMW+3wcz76zCrfMp+yt9ih60NHuXCz6o8cJOScdKqCZNDO8Q0tq9F2jMxpV2f96YduQ6UeAmWJxiHnDvxqOx9LNrMddpdCyAwgWrwaq7raV5/PRq/OUtTLNWtIYSMg+6ckf0UEY4GUtQ8gbJR7CBRcWgbYjlm7eZkrt1KzNXDgCidqEzgNDT1ySt/eLkR/v57yElZ8sEzKJkx+uiB9SRVIBbLcwfqMvRywQtfyPvKG4=~-1~-1~-1; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737415929496.9%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _ga_70947XW48P=GS1.1.1737412981.1.1.1737415929.55.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=a5036154-6256-48a0-baa1-6d1d4ead4ef9' ,
   'dnt': '1' ,
   'origin': 'https://www.tokopedia.com' ,
   'priority': 'u=1, i' ,
   'referer': urlb ,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"' ,
   'sec-ch-ua-mobile': '?0' ,
   'sec-ch-ua-platform': '"Windows"' ,
   'sec-fetch-dest': 'empty' ,
   'sec-fetch-mode': 'cors' ,
   'sec-fetch-site': 'same-site' ,
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36' ,
   'x-price-center': 'true' ,
   'x-source': 'tokopedia-lite' ,
   'x-tkpd-lite-service': 'zeus' ,
   'x-version': '1c2c0dd'}
queb = f'[{{"operationName":"productReviewList","variables":{{"productID":"1624334615","page":1,"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n      }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n      }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n      }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n      }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n      }}\\n      badRatingReasonFmt\\n      __typename\\n    }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n    }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'

In [101]:
#situs c
turlc =  'https://gql.tokopedia.com/graphql/productReviewList'
headc = { 'accept': '*/*' ,
   'accept-language': 'en-US,en;q=0.9' ,
   'bd-device-id': '7462131835184662033' ,
   'content-type': 'application/json' ,
   'cookie': '_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQBJ9OcGCHKWmUAQAAD3L+hRp/QT8vjTsgrbBZ5HS3EOHVKh5GKeTybqhtIBGY/XVGwcatr669ugRz0hepTftu/NbWvUO2Kj7Q9Z01mC9KIDWhN/QNRGiMxbWynQ+qYvjGkKzrlgCaIC7DL45uUd1tB/XI95GaZ4259TRvGsKmOVaA34zhkM+/oiLiPc6oY8rR1/6yTQNlh1zczaE93WjKGosbUdkE2SYt659RWL5FMwQCJ8HGGsj/VI+6dcuzZVwklA==~1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQCOnOF6exDoWUAQAA54D+hRo7rcYdyWKCymwMXzaP7HwNEPgrDEuRSaF519576hoiPLmx0FPlK6IIWkFg5LJAS+6743VHjN+bxYYnezxjHSGsFnsRa/kSij0lP9iETUT7VRD2BFDEbiiCYaS13AHEaADsYxf6bB4i73YmaG6e0tLw/ox92OJUliJ3WsB2zpVOmyTRTCJVh7OFOHl/onLXDBJpGo9WTPjifXMCBvcSStbld0kQMXuT7rwOocRclOOoQ2Ltp7UHA3R+X5vtG+LnYzg/x8SdSnWVXDLv5uLy5v4fY1qihi2PYTxRlt3whXXxdlN8t8X5/mLqf2bMQuJj3Y1u3kmrP5miU9SpZfskrDGgU8QeVCXycK4Vv5eL0lzjhVPI++BlCfhvt45IXH7NoABweLDEqhzN2g+gsI/Hq99pZCLAvEEJMo86IQb7pqo++yq5xm1+PwaIjWFIuRQB+bsZ1IkR3BMTh2v1ke1P4oTT/54sBKLuz0c39NzNEBKkk2uJuBTneIpE1/V6j8AqcAZwhMHwCAMe8GvwRk3S6YSz4lGJgh3wGhbmUnYI~3622200~3552837; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQ5i2pyqIHG2mUAQAAhh0Phg3gcdyujPqux1AKOb2fB96tgAMucrNLZQWocTsgI+tOxBnoDsDJmksfeDUyNbGpPZrOFzbJ/fUxTFNPxtqGhIjrfVHekV9WcwFsk8oV81DFOEvxJ7TZMXkplEs129HNAcwo3bEdCALjW3nT3t/8yGQRl5PHe6eXoHX4kIQXYGPZkoFbDQjIQwaYM4T0lrqX+Im+msBvP7o6WBrtGfY7rA61OshvcobQ/MU7iUaMYA6R1NHQsQ0Zs/Ev79RPFF04oq4eEnqrgKBkzaAdjJplPv57xJ2VTIbEGLwke/eEWDN+lSLVGZl+vKUnz0TkQtsEKhDDwuFflOgYKyngQSybgyzWs0ErbsZFEeSudgn/wvS3phG4qoOdlhrq18yYEh7xMZQNUWx/YzVbO6dMQ1WqWHP9Cjo/z7w7rAbvxSAe8t6VvMMBkiiAgPYWhuYfQzIC1OyNvcRKOso/5/tFuMW+3wcz76zCrfMp+yt9ih60NHuXCz6o8cJOScdKqCZNDO8Q0tq9F2jMxpV2f96YduQ6UeAmWJxiHnDvxqOx9LNrMddpdCyAwgWrwaq7raV5/PRq/OUtTLNWtIYSMg+6ckf0UEY4GUtQ8gbJR7CBRcWgbYjlm7eZkrt1KzNXDgCidqEzgNDT1ySt/eLkR/v57yElZ8sEzKJkx+uiB9SRVIBbLcwfqMvRywQtfyPvKG4=~-1~-1~-1; AMP_TOKEN=%24NOT_FOUND; _dc_gtm_UA-126956641-6=1; _dc_gtm_UA-9801603-1=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737416671574.11%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _ga_70947XW48P=GS1.1.1737412981.1.1.1737416671.41.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=a7290a99-02a2-4e60-9cba-44a01c5d3715' ,
   'dnt': '1' ,
   'origin': 'https://www.tokopedia.com' ,
   'priority': 'u=1, i' ,
   'referer': urlc ,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"' ,
   'sec-ch-ua-mobile': '?0' ,
   'sec-ch-ua-platform': '"Windows"' ,
   'sec-fetch-dest': 'empty' ,
   'sec-fetch-mode': 'cors' ,
   'sec-fetch-site': 'same-site' ,
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36' ,
   'x-price-center': 'true' ,
   'x-source': 'tokopedia-lite' ,
   'x-tkpd-lite-service': 'zeus' ,
   'x-version': '1c2c0dd' }
quec = f'[{{"operationName":"productReviewList","variables":{{"productID":"7576433902","page":1,"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n      }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n     }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n     }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n      }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n      }}\\n      badRatingReasonFmt\\n      __typename\\n    }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n    }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'

In [104]:
#situs D
turld = 'https://gql.tokopedia.com/graphql/productReviewList'
headd = {'accept': '*/*' ,
   'accept-language': 'en-US,en;q=0.9' ,
   'bd-device-id': '7462131835184662033' ,
   'content-type': 'application/json' ,
   'cookie': '_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; AMP_TOKEN=%24NOT_FOUND; _gat_UA-9801603-1=1; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQEunOF5QG9XSUAQAAQYsnhg0mj05Y9Ang4qM7rjJrlPk3ON2Z0vVmNhUW4miM6RTVvT+NX6UCRPmFVm15V3r/RDnPIJTKSK8iu9Onw0VEtun0whzFLhIduU/ZISxnEiO8BfCNOwJex37wv/p5S3LALpsFdX93MNfpXJy1cpXtYNV/SIY7G2hLGQf66KXojR/+3uTSVA0XUrSqOYQrC2w49T1vydH2YhNJP1dyuyV2W0DrW6OirhXT8lQndyMnDtQzj2CZTMnTnTlZ1LOki44NEwfxbwAZ69awNWFX4cSVznCgMitEW1uSd2ND5eCNcmZcuHA77kVLK9aKaFZTuL1K9hbqObR6q3GV6EQhZ6uwmT5eU17Ge28mK0t591RpIeZ4I35O6WFa8eH/zr4Px474UCcigvOFlNEzRDBJXhAIdfpMsTPLyppy6OJq0ejbcnK+fBif5qe5ClsYOaYpAfvl0D/yCXYT1Rj8PiDNRi/0wYLNLkf5Ud10VUOc3tYB54mNda6Dp6wMemsata20eD3Ies94wqaTIjzl6c2342zPZzkCd1VzGwWTM/8BOqk4Nd6v7o1zzK9jd5upXelhrYAqa5HNrdfxAjAHj34DNwosSvs14Lg02cPvtBAm2ZxbtbdcWvmmgbQF7B1EjKZEaHbx4sdgCW6Ohk8=~-1~-1~-1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQEunOF5UG9XSUAQAAQYsnhho4YAAj/sWPjXPOtJsEFS57OSLbkSqZJfLozRp3bZE2AZDs4NoE2K0ogtWNosqXqKFB7gZSsfBRhpRGyuq+xObAkjfZuREjA7YazmLZlyB3JOpUdSrMhBII1mx5E0kOI1DpwaoXuLh0GFXnMq+vo5HgBRJdnQnbXJMjj5C3GAYmOJb/o/RyZ2K9CU/8zbGe7r5mbenr0gF0SvGyNjYTzXbLeci6CU8eDWigM8yFPCv5R2OVoLRqwNUGrjiyjvLEs2c/S+17CZglH/2XK0vNlzL5QL4NgQe/Grh0zG43dOmKUQditnkSte4BhcIzE40JaGQDMkr2OguJ2yN4HTzK8QQN1WH/JeR0bLFVW/I1Jyc9cNCyACtnUf3oc3gawrmjM3s/LIW06ohyOAJRZqdQtqVdXiNVjq5OwgMEmHA5Loau+XW58N+lNjN+Pm0ZmWXsBrkNv3F1IKLcUYZiAutbh6xiaKf73boA9tHzw66/hzRpqhY1nHsktyHQdW6Eh4TYQNsKIVL9UKoGWxAHsnlL34H7/IrY3yK4hwYEfKVNexrgHA==~3622200~3552837; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQ1i2pyq+0+WiUAQAA8SQohhq+dBN7E22saUZIqzDKq5JEYOnOTTwgG7co9xZo2clMvQxsVR5S1HewinhBJdl4HqX7VgHyrqrLc6P7qqp54A+oYlYgwlkhbTNZg5zONB3iHPM5ViSgcmz6Ok6/htU+4I5Dyz5rQqN1RLbKusniDdvpeWSIzHBUPRejwbUuhjlFHR1eMXOhZ5zpngkDJpDzk987UKo2AsPi/Ve1WUecFYJ68Vl5KIZf8NxKULpMWMaPSw==~1; _dc_gtm_UA-9801603-1=1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737417566956.15%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _ga_70947XW48P=GS1.1.1737412981.1.1.1737417566.60.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=9b5829fe-2023-41d2-92be-57c4ad494f38' ,
   'dnt': '1' ,
   'origin': 'https://www.tokopedia.com' ,
   'priority': 'u=1, i' ,
   'referer': urld ,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"' ,
   'sec-ch-ua-mobile': '?0' ,
   'sec-ch-ua-platform': '"Windows"' ,
   'sec-fetch-dest': 'empty' ,
   'sec-fetch-mode': 'cors' ,
   'sec-fetch-site': 'same-site' ,
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36' ,
   'x-price-center': 'true' ,
   'x-source': 'tokopedia-lite' ,
   'x-tkpd-lite-service': 'zeus' ,
   'x-version': '1c2c0dd' }
qued = f'[{{"operationName":"productReviewList","variables":{{"productID":"1624323568","page":1,"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n    }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n      }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n    }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n    }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n    }}\\n      badRatingReasonFmt\\n      __typename\\n   }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n   }}\\n    hasNext\\n    totalReviews\\n    __typename\\n }}\\n}}\\n"}}]'

In [137]:
#situs E
turle = 'https://gql.tokopedia.com/graphql/productReviewList'
heade =  {'accept': '*/*' ,
   'accept-language': 'en-US,en;q=0.9' ,
   'bd-device-id': '7462131835184662033' ,
   'content-type': 'application/json' ,
   'cookie': '_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; AMP_TOKEN=%24NOT_FOUND; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQE+nOFwmjoHWUAQAA0xEshg3H/DdlfkA3HYlkRbz5hdwp2vDzw0WTBBniSd9lP/qDmFSOd5AmA+33hyUaAyGhy9syP67J1J+qF6Xp+7Jzb7b6UELM839g3k6QQQl9l2UVx1/ibR7OGaloWv0/jFqnKeWTdpkZNCRsTcCS7+sPXZexf21Nfmd1f/vwm7G1qNvSAfBc2zlFeHRfDc7vW+NK3jwG/fPNoUeoLloJJT2FlPodcHtw7SH32A4Ijfg0UFJeUTEIhY9iLUCThYf78vZOR630FGwAmdXY6tHfRHsudAQWuHXgi9Ek1b/KjKd27CTdPqNWwB+JcDYoaZLKKFzZlcGnjNdIRhRKuAli1Mu1EDPUQLuFEAgErSQcJM7id3Uwit6pFWjhJF/mAAXmffzxVfXzC6cIbi720gHaiCT+ulgpyf7XAhnZSLLTMVtZJC/SC06DqxzIPtMLWt0D5cbPu4VjQ1IzxcTrWFdBZLVRfLeAWBTuMeT3VuMWXtFBcuV7uTMXUoyqMLGa1OaoNQdsc6cX/gyFW+qEIQYyurY6ZNObMBHDRzz739JfJ1ng/ov5QTxgbyNuxCUpF7O6S1YL22a1HX+kktRHvnYgELAWzw6VDf/ACV3t3gul3Tvlvh1R66Dpp+JiWaaUJqyN1lkwrEUrUZCfPDo=~-1~-1~-1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQE+nOFwqjoHWUAQAA0xEshhocqAjR5TwM/b6DYUlkifT4002HQFcNQFjs+LV4zUtgLs62zYpUbo1s46fv/zc7mul7xi9r7EtL8P7YNMXcAJUvl87y/6wfW/WtsbysxzQFlu5idYnf6p7Fir8rusCcEfKnt+sSGEFsnsjem+ydZJGA2HA4ba1P0Y1dzXwcidZJJw9w6epFgvNXQshN/KPsF4pb1Z51AjB6ym38aMER74yVWjhSg/kS2YVHGvyQIiPxB7TbmRaSZGl3gkHY9Tr0/okvdN7U7SDgt/S9kxM2/lIpl+nml2k8zF/U9tFYjm9VK5a+5TNBAl6C+Jz5DXILJyG/QWAH7SoMfIsOjyYGFtUyQ9ruGm3EfwunMXDT5tWk0SY/ekveIjhQQtRxns62IaxxBUR0lvqk3uOuuvfkVKQyPgLheWPKvnPJTca+Tkik/KE5pCTFFIsGy296SZUMKe15M0Lgr1JadBmI1kAerOLrUzLwTjyxGo5JB0jAQQZ2aRpIbViye7hvEoGpARdJ83KnfzPav/dK11PIW3Mzh79bLD1rAG6ERXqTdg2emiicjrSrlg==~3622200~3552837; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQ1i2pyqXJ+WiUAQAAGE0uhhpH+Z8MdDCA+HoQmXLw8EjPpCzFi/kzBSe1Sx8fyk2NZtcjdHwb05U9UlhMcUUgABGWC5BuWRYsDk74g46VSqEAgjrBifU6/Pfx18isdxS44HAfsxBVnT36CQrPoSEX54QpvImZ+4OBwDqNmsby6+E8LP+wNnrw6AKG8wEnaD4+f8ig+r2NPyKO7VpN/izStYkyotRO0lXFt/C1cmwA+FKHCIW5Ga7y1lzwkydAKMxT5A==~1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737419141703.17%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737412981.1.1.1737419141.60.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=35155137-d279-46fd-9f93-2cd0e963e08e' ,
   'dnt': '1' ,
   'origin': 'https://www.tokopedia.com' ,
   'priority': 'u=1, i' ,
   'referer': urle ,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"' ,
   'sec-ch-ua-mobile': '?0' ,
   'sec-ch-ua-platform': '"Windows"' ,
   'sec-fetch-dest': 'empty' ,
   'sec-fetch-mode': 'cors' ,
   'sec-fetch-site': 'same-site' ,
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36' ,
   'x-price-center': 'true' ,
   'x-source': 'tokopedia-lite' ,
   'x-tkpd-lite-service': 'zeus' ,
   'x-version': '1c2c0dd'}
quee = f'[{{"operationName":"productReviewList","variables":{{"productID":"1624323568","page":1,"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n      }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n     }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n     }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n     }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n     }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n    }}\\n      badRatingReasonFmt\\n      __typename\\n   }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n   }}\\n    hasNext\\n    totalReviews\\n    __typename\\n  }}\\n}}\\n"}}]'

In [142]:
#Situs F
turlf = 'https://gql.tokopedia.com/graphql/productReviewList'
headf = { 'accept': '*/*' ,
   'accept-language': 'en-US,en;q=0.9' ,
   'bd-device-id': '7462131835184662033' ,
   'content-type': 'application/json' ,
   'cookie': '_UUID_NONLOGIN_=8f99d5dd839ad1780723e8595ad38987; _UUID_NONLOGIN_.sig=ciTgSOoFDossyyjCLdfGFWxNt2w; _gcl_au=1.1.331787603.1737412981; _SID_Tokopedia_=rE4HKUspgx3YEb4M6zHuh7FPxz7hEsGH2ilCP1heDP2T_SJ2uXv4d-NBiIN2v_xzAm39GiewdOM2V8GPEuWJttUV1m23WNsCqBNtPOTd-3gHZKhh1YOgff3lIHo1fIeO; DID=3c43b6b28f92625d9d7b390ba99c537c62a4d87462943c414f7eab7fe94efcc3e7886ac9c2aafbef560d9fde2b03553a; DID_JS=M2M0M2I2YjI4ZjkyNjI1ZDlkN2IzOTBiYTk5YzUzN2M2MmE0ZDg3NDYyOTQzYzQxNGY3ZWFiN2ZlOTRlZmNjM2U3ODg2YWM5YzJhYWZiZWY1NjBkOWZkZTJiMDM1NTNh47DEQpj8HBSa+/TImW+5JCeuQeRkm5NMpJWZG3hSuFU=; _gid=GA1.2.1786514285.1737412982; ak_bmsc=2370BD7F56AE72218B590E7172AFB6E0~000000000000000000000000000000~YAAQ5i2pyiq8GmmUAQAAqzjihRqEn1r+H8D382KNWkQF2Q5qQrV56FrBezdinISsra53rNILM2p7daQBGJhXVkQ/8NZhkZR/BsDP75R7yFtjEM0Rfjz0Tk/YpKszleH3jUNiaJT2qYkya/MH3o7st6fdrWBnZ1OtMWwipJXnpGyhJjcfbcgivGC3pbBNjtJQNZoRo0V3elQcSBAsJBvh7MtEo9vHEDxWyrL/f+a4V8+nbJih3KV+ANthkLcYWaSf7RA6ThQ57CgwcXyn5chcztUHYoJneAwI/Spa1sj6xR5ZqTRb4iaKSPblRxWY98itGrEhxasWFmsTgzhc7Sa9V8Smm9wkTkCAGKJQuPxx7l8ozJ7wLAlPIUkNequHEoVgpYrlgXYa8APW25wsEMgqCzeEqXQ4cxDTJQ+CKdl1DillrL3qTpNDj37JLcs5uMx4eOOuAmAcBVlYHMa7HKBB; _tt_enable_cookie=1; _ttp=9ko8J_DU5G9xwpTPQmCcci2-EAb.tt.1; TOPATK=MkwobnZbQCujAywkR-WEmg; l=1; tuid=39275554; gec_id=168687219962298048; uidh=hQ343xlJhV8WFIBvzr8NcCaECmZcWRGgONmdJbMgjPA=; g_yolo_production=1; _hjSessionUser_714968=eyJpZCI6ImZhYjYyOGQ2LTFhMmQtNWM2NS1iY2E0LWI3ZmVkMTQ2YjY4MSIsImNyZWF0ZWQiOjE3Mzc0MTMwMDc5NTQsImV4aXN0aW5nIjpmYWxzZX0=; FPF=1; uide=RKzymP3PuU0ExKH5fb+TNGxXZ1glgdsMuHH0FJ3NktJuXxpF; aus=1; _CASE_=2e77341c31776f776361606064666d61777977361c31776f64626d7977311c31776f67676c67797739002531776f77676567607865647867640165606f61666f60637e65626f6565777977393421776f7778637b6767616265636c777977393739776f77383439343e3475363a3b21272c7530262134213075223c2734777977393a3b32776f776465637b6c616564626077797725163a776f776466616365777977261c31776f6464606665606266797726012c2530776f773a3a36777977221c31776f657977223d26776f770e08777977223d2626776f770e087728; _UUID_CAS_=9e9d1a0a-09c5-419f-a24b-dd1adcdbf1e6; _fbp=fb.1.1737413098523.68838001380503701; AMP_TOKEN=%24NOT_FOUND; _abck=5BD2CFC94D1BBDB7FBD23F6C7830515B~0~YAAQE+nOFwmjoHWUAQAA0xEshg3H/DdlfkA3HYlkRbz5hdwp2vDzw0WTBBniSd9lP/qDmFSOd5AmA+33hyUaAyGhy9syP67J1J+qF6Xp+7Jzb7b6UELM839g3k6QQQl9l2UVx1/ibR7OGaloWv0/jFqnKeWTdpkZNCRsTcCS7+sPXZexf21Nfmd1f/vwm7G1qNvSAfBc2zlFeHRfDc7vW+NK3jwG/fPNoUeoLloJJT2FlPodcHtw7SH32A4Ijfg0UFJeUTEIhY9iLUCThYf78vZOR630FGwAmdXY6tHfRHsudAQWuHXgi9Ek1b/KjKd27CTdPqNWwB+JcDYoaZLKKFzZlcGnjNdIRhRKuAli1Mu1EDPUQLuFEAgErSQcJM7id3Uwit6pFWjhJF/mAAXmffzxVfXzC6cIbi720gHaiCT+ulgpyf7XAhnZSLLTMVtZJC/SC06DqxzIPtMLWt0D5cbPu4VjQ1IzxcTrWFdBZLVRfLeAWBTuMeT3VuMWXtFBcuV7uTMXUoyqMLGa1OaoNQdsc6cX/gyFW+qEIQYyurY6ZNObMBHDRzz739JfJ1ng/ov5QTxgbyNuxCUpF7O6S1YL22a1HX+kktRHvnYgELAWzw6VDf/ACV3t3gul3Tvlvh1R66Dpp+JiWaaUJqyN1lkwrEUrUZCfPDo=~-1~-1~-1; bm_sz=5AFD8A5B2276517CD536C357A5D9F68D~YAAQE+nOFwqjoHWUAQAA0xEshhocqAjR5TwM/b6DYUlkifT4002HQFcNQFjs+LV4zUtgLs62zYpUbo1s46fv/zc7mul7xi9r7EtL8P7YNMXcAJUvl87y/6wfW/WtsbysxzQFlu5idYnf6p7Fir8rusCcEfKnt+sSGEFsnsjem+ydZJGA2HA4ba1P0Y1dzXwcidZJJw9w6epFgvNXQshN/KPsF4pb1Z51AjB6ym38aMER74yVWjhSg/kS2YVHGvyQIiPxB7TbmRaSZGl3gkHY9Tr0/okvdN7U7SDgt/S9kxM2/lIpl+nml2k8zF/U9tFYjm9VK5a+5TNBAl6C+Jz5DXILJyG/QWAH7SoMfIsOjyYGFtUyQ9ruGm3EfwunMXDT5tWk0SY/ekveIjhQQtRxns62IaxxBUR0lvqk3uOuuvfkVKQyPgLheWPKvnPJTca+Tkik/KE5pCTFFIsGy296SZUMKe15M0Lgr1JadBmI1kAerOLrUzLwTjyxGo5JB0jAQQZ2aRpIbViye7hvEoGpARdJ83KnfzPav/dK11PIW3Mzh79bLD1rAG6ERXqTdg2emiicjrSrlg==~3622200~3552837; bm_sv=23F216B8BDF571EE83DBA5B0205B1B79~YAAQ1i2pyqXJ+WiUAQAAGE0uhhpH+Z8MdDCA+HoQmXLw8EjPpCzFi/kzBSe1Sx8fyk2NZtcjdHwb05U9UlhMcUUgABGWC5BuWRYsDk74g46VSqEAgjrBifU6/Pfx18isdxS44HAfsxBVnT36CQrPoSEX54QpvImZ+4OBwDqNmsby6+E8LP+wNnrw6AKG8wEnaD4+f8ig+r2NPyKO7VpN/izStYkyotRO0lXFt/C1cmwA+FKHCIW5Ga7y1lzwkydAKMxT5A==~1; _dc_gtm_UA-126956641-6=1; ISID=%7B%22www.tokopedia.com%22%3A%22d3d3LnRva29wZWRpYS5jb20%3D.85eb825b3d925425e8af813e0d7c97b5.1737412981788.1737412981788.1737419549421.17%22%2C%22accounts.tokopedia.com%22%3A%22YWNjb3VudHMudG9rb3BlZGlhLmNvbQ%3D%3D.8e22ef5e53b67456ebb5f5f1684e5283.1737413008027.1737413008027.1737413033102.1%22%7D; _dc_gtm_UA-9801603-1=1; _ga_70947XW48P=GS1.1.1737412981.1.1.1737419549.60.0.0; _ga=GA1.1.1911615655.1737412982; webauthn-session=7a923bdd-32cb-45cb-bf37-b8f374c4ccac' ,
   'dnt': '1' ,
   'origin': 'https://www.tokopedia.com' ,
   'priority': 'u=1, i' ,
   'referer': urlf ,
   'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"' ,
   'sec-ch-ua-mobile': '?0' ,
   'sec-ch-ua-platform': '"Windows"' ,
   'sec-fetch-dest': 'empty',
   'sec-fetch-mode': 'cors' ,
   'sec-fetch-site': 'same-site' ,
   'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36' ,
   'x-price-center': 'true' ,
   'x-source': 'tokopedia-lite' ,
   'x-tkpd-lite-service': 'zeus' ,
   'x-version': '1c2c0dd' }
quef = f'[{{"operationName":"productReviewList","variables":{{"productID":"2390937942","page":1,"limit":10,"sortBy":"informative_score desc","filterBy":""}},"query":"query productReviewList($productID: String\u0021, $page: Int\u0021, $limit: Int\u0021, $sortBy: String, $filterBy: String) {{\\n  productrevGetProductReviewList(productID: $productID, page: $page, limit: $limit, sortBy: $sortBy, filterBy: $filterBy) {{\\n    productID\\n    list {{\\n      id: feedbackID\\n      variantName\\n      message\\n      productRating\\n      reviewCreateTime\\n      reviewCreateTimestamp\\n      isReportable\\n      isAnonymous\\n      imageAttachments {{\\n        attachmentID\\n        imageThumbnailUrl\\n        imageUrl\\n        __typename\\n     }}\\n      videoAttachments {{\\n        attachmentID\\n        videoUrl\\n        __typename\\n    }}\\n      reviewResponse {{\\n        message\\n        createTime\\n        __typename\\n    }}\\n      user {{\\n        userID\\n        fullName\\n        image\\n        url\\n        __typename\\n    }}\\n      likeDislike {{\\n        totalLike\\n        likeStatus\\n        __typename\\n    }}\\n      stats {{\\n        key\\n        formatted\\n        count\\n        __typename\\n    }}\\n      badRatingReasonFmt\\n      __typename\\n  }}\\n    shop {{\\n      shopID\\n      name\\n      url\\n      image\\n      __typename\\n  }}\\n    hasNext\\n    totalReviews\\n    __typename\\n }}\\n}}\\n"}}]'

## Persiapan respon

In [143]:
#pembuatan respon
respa = requests.post(turla,headers=heada,data=quea) #dari produk a
respb = requests.post(turlb,headers=headb,data=queb) #dari produk b
respc = requests.post(turlc,headers=headc,data=quec) #dari produk c
respd = requests.post(turld,headers=headd,data=qued) #dari produk d
respe = requests.post(turle,headers=heade,data=quee) #dari produk e
respf = requests.post(turlf,headers=headf,data=quef) #dari produk f
#ke produk

In [144]:
#inspeksi teks
print("isi respon dari respa:", respa.text)  # Isi dari 'respa'
print("isi respon dari respb:", respb.text)  # Isi dari 'respb'
print("isi respon dari respc:", respc.text)  # Isi dari 'respc'
print("isi respon dari respd:", respd.text) # Isi dari 'respd'
print("isi respon dari respe:", respe.text) # Isi dari 'respe'
print("isi respon dari respf:", respf.text) # Isi dari 'respf'

isi respon dari respa: [{"data":{"productrevGetProductReviewList":{"productID":"1822509697","list":[{"id":"1096280412","variantName":"Jazlyn Black - XXL","message":"barang bagus kualitas nya top harga nya naik overall 100 dijamin ori","productRating":5,"reviewCreateTime":"1727090981","reviewCreateTimestamp":"4 bulan lalu","isReportable":true,"isAnonymous":true,"imageAttachments":[{"attachmentID":"135027678","imageThumbnailUrl":"https://images.tokopedia.net/img/cache/200-square/bjFkPX/2024/9/23/22551dc4-2752-4ffe-9283-c63f65a33d95.jpg","imageUrl":"https://images.tokopedia.net/img/cache/600/bjFkPX/2024/9/23/22551dc4-2752-4ffe-9283-c63f65a33d95.jpg","__typename":"reviewAttachmentPDPType"},{"attachmentID":"135027679","imageThumbnailUrl":"https://images.tokopedia.net/img/cache/200-square/bjFkPX/2024/9/23/50b9df64-9576-48c7-8b4a-0ab6891e3144.jpg","imageUrl":"https://images.tokopedia.net/img/cache/600/bjFkPX/2024/9/23/50b9df64-9576-48c7-8b4a-0ab6891e3144.jpg","__typename":"reviewAttachmentPDP

In [ ]:
#inspeksi hasil json
produka = respa.json()[0]['data']['productrevGetProductReviewList']['list']
produkb = respb.json()[0]['data']['productrevGetProductReviewList']['list']
produkc = respc.json()[0]['data']['productrevGetProductReviewList']['list']
produkd = respd.json()[0]['data']['productrevGetProductReviewList']['list']
produke = respe.json()[0]['data']['productrevGetProductReviewList']['list']
produkf = respf.json()[0]['data']['productrevGetProductReviewList']['list']
print(produkf)

# Persiapan ke file csv

In [ ]:
#penyatuan hasil json
jserigo = produka + produkb + produkc + produkd + produke + produkf #sampel dari produk erigo
#cek hasil
print(jserigo)

In [148]:
#konversi ke bentuk csv
namacsv = 'dfori.csv' #nama yang diinginkan
#fungsi konversi
with open(namacsv, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = [
        'id', 'variantName', 'message', 'productRating',
        'reviewCreateTime', 'isAnonymous', 'user_fullName',
        'user_url', 'totalLike'
    ]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    # Write header row
    writer.writeheader()

    # Write each merged review to the CSV
    for review in jserigo:
        writer.writerow({
            'id': review['id'],
            'variantName': review.get('variantName', ''),
            'message': review.get('message', ''),
            'productRating': review.get('productRating', ''),
            'reviewCreateTime': review.get('reviewCreateTime', ''),
            'isAnonymous': review.get('isAnonymous', ''),
            'user_fullName': review['user'].get('fullName', '') if review.get('user') else '',
            'user_url': review['user'].get('url', '') if review.get('user') else '',
            'totalLike': review['likeDislike'].get('totalLike', '') if review.get('likeDislike') else ''
        })

print(f'Merged reviews saved to {namacsv}')

Merged reviews saved to dfori.csv


In [151]:
#inspeksi sampel
dforif = pd.read_csv('dfori.csv')
dfkopi = dforif.copy()
#ke inspeksi sampel

In [152]:
kepala = dfkopi.head(7)
kepala

,id,variantName,message,productRating,reviewCreateTime,isAnonymous,user_fullName,user_url,totalLike
0,1096280412,Jazlyn Black - XXL,barang bagus kualitas nya top harga nya naik o...,5,1727090981,True,m***l,NaN,1
1,1131666530,Jazlyn Black - XXL,"modelnya keren, bahannya bagus cuma agak kepan...",5,1734432169,False,Ahmad,https://www.tokopedia.com/people/12344984,1
2,1130273219,Jazlyn Black - L,"tinggi 175 bb 60 ukuran L cocok si, pas dan en...",5,1734122573,False,Syah,https://www.tokopedia.com/people/142632910,0
3,757190522,L,enak di pake adem dan kece pas di badan gw. ta...,5,1663576439,True,S***a,NaN,0
4,1016018259,Jazlyn Black - L,APAAN INI WOY KEREN BANGET KEMEJA NYA ga usa r...,5,1710777365,False,raden,https://www.tokopedia.com/people/210403877,1
5,816486546,XXL,"oke paket di terima dengan baik ,, untuk kuali...",5,1671820166,False,Ostoee,https://www.tokopedia.com/people/219595677,0
6,773225440,L,"pas, bahannya enak banget dibadan dan juga ker...",5,1665743375,False,Indrath,https://www.tokopedia.com/people/91590875,0


# Tahap Pembersihan data

## Inspeksi dan pembersihan tahap awal

In [153]:
#informasi df
dfkopinfo = dfkopi.info()
dfkopinfo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                60 non-null     int64 
 1   variantName       59 non-null     object
 2   message           60 non-null     object
 3   productRating     60 non-null     int64 
 4   reviewCreateTime  60 non-null     int64 
 5   isAnonymous       60 non-null     bool  
 6   user_fullName     60 non-null     object
 7   user_url          32 non-null     object
 8   totalLike         60 non-null     int64 
dtypes: bool(1), int64(4), object(4)
memory usage: 3.9+ KB
